In [53]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your OpenAI API key: ')
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Langchain API key: ")

os.environ["LANGCHAIN_TRACING_V2"]= "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="RAG-test"

from langchain_openai import ChatOpenAI, OpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=os.environ.get("OPENAI_API_KEY"))

In [56]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate



In [30]:
import os

# Specify the absolute path to the target file
target_file_path = r"\Users\mava08\OneDrive - AF Gruppen ASA\Datamateriale-endringer\CCB NS 8415 Underentreprise, avklarende kontraktsm_te, referat.pades.pdf"

# Get the current working directory
current_directory = os.getcwd()

# Find the relative path from the current directory to the target file
relative_path = os.path.relpath(target_file_path, start=current_directory)

print(f"The relative path to the file is: {relative_path}")

The relative path to the file is: ..\..\..\..\OneDrive - AF Gruppen ASA\Datamateriale-endringer\CCB NS 8415 Underentreprise, avklarende kontraktsm_te, referat.pades.pdf


In [31]:
loader = PyPDFLoader(relative_path)
pages = loader.load_and_split()

print(pages[0])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())



page_content='Identifikasjon  Rev:  Parefere : Side  \nCCB NS 8415 Underentreprise, avklarende kontraktsmøte, referat  07/28.02.2020  1 av 17 \n \n \n \nMØTEREFERAT  \n \nMøte type:            Avklarende kontraktsmøte  NS 8415   \nMøte nr. : 01/01 \n \nMøtedato : 02.10.2023   \nSted:  Ulvenveien 91 \n \nMøteleder:  Ola Randby   \nReferent: Ola Randby  \n \nProsjekt /nr.:  Construction City / 14100   \n \n \nDeltakere:    \nNavn   \nFirma / Funksjon   \nAnm .: \n \n Ola Randby  CCB  – Prosjektleder drift fase 2   \n Petter Tufte  \n \nRoger H årklau  \n CCB – Anleggsleder  plan  2-11 \n  \nCCB – Anleggslede r plan U2 -1  \n Haakon N. Kristiansen  \n \nStig Mathisen  \n \nHans Petter Berg  \n CCB – Prosjektingeniør  \n \nOBI – Markedssjef  \n \nOBI – Prosjektleder   \n    \n(X) = Ikke tilstede  \n \nReferat til:  \nDeltagere og stedfortredere.   \nKopi til:  \nCCB  \n \n \nAvklaring med f oretak:  \n \nFag / leveranse:  \n Himling og Systemvegg er \nForetak s \nnavn/nummer  Oslo B ygginn

In [68]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
# Prompt
template = """Du er en assistent som skal sjekke om den innkommende etterspørselen/endringen fra et selskap med krav om betaling er gyldig i henhold til relevante kontraktsdokumenter, eller om arbeidet egentlig er inkludert i prising gitt i kontrakten. Begrunn hvorfor/hvorfor ikke den innkommende etterspørselen/endringen er gyldig i henhold til kontraktsdokumentene.:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)



rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


rag_chain.invoke("""Tittel
El.kanal gjennom himling

Beskrivelse av forholdet
Det skal monteres el-kanal gjennom himlingene

Utredning og forslag til løsning
Kappet vegglister for el-kanalene i bygg F 5-6-7-8 etg

Vederlagsjustering etter
 Regningsarbeid

Kostnadsoversikt
Postnr.	Beskrivelse	Firma	Enhet	Mengde	Enhetspris	Påslag	Totalbeløp	
1	arbeid	OBI	timer	8	660	-	5 280	
Totalsum alle kostnader	5 280
Totalsum godkjente kostnader	0
Totalsum avslåtte kostnader	5 280
""")

'Etterspørselen om å montere el-kanal gjennom himlingene i bygg F 5-8 etasje er ikke gyldig i henhold til kontraktsdokumentene. Kontrakten spesifiserer at tilpasninger til tekniske produkter, som elektriske brystningskanaler, er inkludert i underentreprenørens enhetspriser. Derfor bør arbeidet med å montere el-kanal gjennom himlingene ikke medføre ekstra kostnader for byggherren. Etterspørselen om vederlagsjustering etter regningsarbeid bør derfor avslås, da arbeidet er inkludert i prisen gitt i kontrakten.'